In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import os
import numpy as np
from tensorflow import keras
from tensorflow.keras import layers

import tensorflow as tf
import tensorflowjs as tfjs

import wandb
from wandb.keras import WandbCallback

import dataset
import schedulers

assert tf.config.list_physical_devices('GPU')

In [ ]:
LETTERS_SIZE = len(dataset.letters_table)
NIQQUD_SIZE = len(dataset.niqqud_table)
DAGESH_SIZE = len(dataset.dagesh_table)
SIN_SIZE = len(dataset.sin_table)

def build_model(units=500, maxlen=64):
    inp = keras.Input(shape=(maxlen,), batch_size=None)
    embed = layers.Embedding(LETTERS_SIZE, units, mask_zero=True)(inp)
    
    layer = layers.Bidirectional(layers.LSTM(units, return_sequences=True), merge_mode='sum')(embed)
    layer = layers.add([layer, layers.Bidirectional(layers.LSTM(units, return_sequences=True), merge_mode='sum')(layer)])
    layer = layers.BatchNormalization()(layer)
    layer = layers.add([embed, layers.Dense(units, activation='relu')(layer)])

    outputs = [
        layers.Softmax(name='N')(layers.Dense(NIQQUD_SIZE)(layer)),
        layers.Softmax(name='D')(layers.Dense(DAGESH_SIZE)(layer)),
        layers.Softmax(name='S')(layers.Dense(SIN_SIZE)(layer)),
    ]
    model = keras.Model(inputs=inp, outputs=outputs)

    return model


In [ ]:

# masked version of accuracy and sce
def accuracy(real, pred):
    acc = tf.keras.metrics.sparse_categorical_accuracy(real, pred)

    mask = tf.cast(tf.math.logical_not(tf.math.equal(real, 0)), dtype=acc.dtype)
    acc *= mask

    return tf.reduce_sum(acc) / tf.reduce_sum(mask)

def sparse_categorical_crossentropy(y_true, y_pred, sample_weight=None):
    loss = tf.keras.losses.sparse_categorical_crossentropy(y_true, y_pred)

    mask = tf.cast(tf.math.logical_not(tf.math.equal(y_true, 0)), dtype=loss.dtype)
    loss *= mask

    return tf.reduce_sum(loss) / tf.reduce_sum(mask) 


In [ ]:
MAXLEN = 64

data = {}
data['mix'] = dataset.load_data([
    'hebrew_diacritized_private/poetry',
    'hebrew_diacritized_private/rabanit',
    'hebrew_diacritized_private/pre_modern'], validation_rate=0.1, maxlen=MAXLEN)

data['modern'] = dataset.load_data([
    'hebrew_diacritized/modern'], validation_rate=0.2, maxlen=MAXLEN)

In [ ]:
%env WANDB_MODE run

UNITS = 500
BATCH_SIZE = 64

model = build_model(units=UNITS, maxlen=MAXLEN)

model.compile(loss=sparse_categorical_crossentropy, optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),
              metrics={'N': accuracy, 'D': accuracy, 'S': accuracy})

model.save_weights('./checkpoints/uninit')

config = {
    'batch_size': BATCH_SIZE,
    'units': UNITS,
    'maxlen': MAXLEN,
    'model': model,
    'batch_size': BATCH_SIZE,
}

config['order'] = [
    ('mix', [(30e-4, 80e-4, 1e-4)], 'mix'),
    ('modern', [(50e-4, 50e-4, 1e-5)], 'modern'),
    ('modern', [(50e-4, 50e-4, 1e-5), (50e-4, 50e-4, 1e-5)], 'modern_over'),
]

run = wandb.init(project="dotter",
                 name='final_eval-nofix',
                 tags=['CLR', 'ordered'],
                 config=config)

def get_xy(d):
    if d is None:
        return None
    x = d.normalized
    y = {'N': d.niqqud, 'D': d.dagesh, 'S': d.sin }
    return (x, y)

with run:
    for kind, clrs, save in config['order']:
        train, validation = data[kind]
        
        training_data = (x, y) = get_xy(train)
        validation_data = get_xy(validation)
        
        wandb_callback = WandbCallback(log_batch_frequency=10, training_data=training_data, validation_data=validation_data,
                                       log_weights=True)
        
        for clr in clrs:
            scheduler = schedulers.CircularLearningRate(*clr)
            scheduler.set_dataset(train, BATCH_SIZE)
            callbacks = [wandb_callback, scheduler]
            history = model.fit(x, y, validation_data=validation_data, batch_size=BATCH_SIZE, verbose=1, callbacks=callbacks)
        model.save(os.path.join(wandb.run.dir, save + ".h5"))
        model.save_weights('./checkpoints/' + save)


In [ ]:
model.load_weights('./checkpoints/modern_over')

model.compile()
model.save('modern.h5')
tfjs.converters.save_keras_model(model, '.')

In [ ]:
import matplotlib.pyplot as plt

fig, ax = plt.subplots(nrows=2, ncols=3)

for n, v in enumerate(['accuracy', 'loss'], 0):
    for n1, t in enumerate(['N', 'D', 'S'], 0):
        p = ax[n][n1]
        p.plot(history.history[t + '_' + v][0:])
        p.plot(history.history['val_' + t + '_' +  v][0:])
        p.legend([t + '_Train', t + '_Test'], loc='center right')

plt.tight_layout()

In [ ]:
model.load_weights('./checkpoints/modern_over')
test = dataset.load_file('test/ModernTestCorpus-HebrewWiki1-simple.txt', maxlen)
x = test.normalized
y = {'N': test.niqqud, 'D': test.dagesh, 'S': test.sin }

model.compile(loss=sparse_categorical_crossentropy,
              metrics={'N': accuracy, 'D': accuracy, 'S': accuracy})

model.evaluate(x=x, y=y, batch_size=BATCH_SIZE)

In [ ]:
model.load_weights('./checkpoints/modern_over')

def real_evaluation(data, s=slice(0, None), print_comparison=True):
    batch = data.normalized[s]
    prediction = model.predict(batch)
    [actual_niqqud, actual_dagesh, actual_sin] = [dataset.from_categorical(prediction[0]), dataset.from_categorical(prediction[1]), dataset.from_categorical(prediction[2])]
    [expected_niqqud, expected_dagesh, expected_sin] = [data.niqqud[s], data.dagesh[s], data.sin[s]]
    actual = dataset.merge(data.text[s], ts=batch, ns=actual_niqqud, ds=actual_dagesh, ss=actual_sin)
    expected = dataset.merge(data.text[s], ts=batch, ns=expected_niqqud, ds=expected_dagesh, ss=expected_sin)
    total_letters = []
    total_words = []
    for i, (b, a, e) in enumerate(zip(batch, actual, expected)):
        res = ( (expected_niqqud[i][b > 16] == actual_niqqud[i][b > 16])
              & (expected_dagesh[i][b > 16] == actual_dagesh[i][b > 16])
              & (expected_sin[i][b > 16] == actual_sin[i][b > 16]) )
        total_letters.extend(res)
        
        for aw, ew in zip(a.split(), e.split()):
            if len([x for x in 'אבגדהוזחטיכלמנסעפצקרשתךםןףץ' if x in aw]) > 1:
                total_words.append(aw == ew)
                if print_comparison and aw != ew:
                    print(aw, ew)
        
        if print_comparison:
            print('מצוי: ', a)
            print('רצוי: ', e)
            print(f'{np.mean(res):.2%} ({len(res)-np.sum(res)} out of {len(res)})')
            print()
    print(f'letters: {np.mean(total_letters):.2%}, words: {np.mean(total_words):.2%}')

real_evaluation(test, print_comparison=False)